<a href="https://colab.research.google.com/github/alxmzr/Colab/blob/main/One_Trading_Strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# One Trading Strategy Every Trader Should Know | Coinmonks
# https://medium.com/coinmonks/the-5-minute-trading-strategy-5a83507a630e

In [ ]:
# pip install python-binance pandas-ta mplfinance numpy pandas

In [ ]:
import pandas as pd
import numpy as np
import pandas_ta as ta
from binance.client import Client
from datetime import datetime, timedelta
import math
import mplfinance as mpf

api_key = 'DLuwcpU67Emo1FtVkiuCTzCFYkUpk4OvdmKGTNb0Wj4bhw3NXxUcQBoQtBVA7BpM'
api_secret = 'g681jmxJ2Dczth4qYaK8TFlqlNgqDkjcQeJCY0BI2CQABerfzYMPBU4JivKKomls'

symbol = 'BTCUSDT'
interval = Client.KLINE_INTERVAL_1DAY

client = Client(api_key, api_secret)

In [ ]:
def getData(symbol, interval, lookback):
  frame = pd.DataFrame(client.get_historical_klines(symbol, Client.KLINE_INTERVAL_1DAY, "1 Jan, 2022", "1 Apr, 2022"))
  frame = frame.iloc[:,:6]
  frame.columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume']
  frame = frame.set_index('Time')
  frame.index = pd.to_datetime(frame.index, unit='ms')
  frame = frame.astype(float)
  return frame

In [ ]:
def calculate_entries(symbol, interval='1d', lookhead=1500):
  SMA_Period = 20
  df = getData(symbol, interval, lookhead)
  key_name = f'SMA_{SMA_Period}'
  max_change_percentage = 0.5
  df.ta.macd(append=True)
  df.ta.sma(length=SMA_Period, append=True)
  df['Change'] = abs(df['Close'].pct_change())
  df['MACD_GREEN'] = df['MACDh_12_26_9'] > 0
  df['MACD_RED'] = df['MACDh_12_26_9'] < 0
  df['MACD_DARK_GREEN'] = df['MACD_GREEN'] &  (df['MACDh_12_26_9'] >  df['MACDh_12_26_9'].shift(1))
  df['MACD_DARK_RED'] = df['MACD_RED'] &  (df['MACDh_12_26_9'] <  df['MACDh_12_26_9'].shift(1))

  df['Entries_Short'] = (df.Open > df[key_name]) & (df['Close'] < df[key_name]) & (df['Close'] < df['Open']) & \
                        (df['Change'] < max_change_percentage) & (df['Low'].shift(1) > df[key_name].shift(1)) & \
                        (df['MACD_DARK_GREEN'] == False) & (df['MACD_12_26_9'] < 0) & (df['MACDs_12_26_9'] < 0)

  df['Entries_Long'] = (df.Open < df[key_name]) & (df['Close'] > df[key_name]) & (df['Close'] > df['Open']) & \
                        (df['Change'] < max_change_percentage) & (df['High'].shift(1) < df[key_name].shift(1)) & \
                        (df['MACD_DARK_RED'] == False) & (df['MACD_12_26_9'] > 0) & (df['MACDs_12_26_9'] > 0)
  return df

In [ ]:
def make_plot(df, long_column='Entries_Long', short_column='Entries_Short', margin=0):
  enter_short = []
  enter_long = []
  for k,row  in df.iterrows():
    if row[short_column]:
      enter_short.append(row.High + margin)
    else:
      enter_short.append(np.nan)

  for k,row  in df.iterrows():
    if row[long_column]:
      enter_long.append(row.Low - margin )
    else:
      enter_long.append(np.nan)

  up_plot = mpf.make_addplot(enter_short, type='scatter', marker='v', markersize=25, panel=0, color='r')
  down_plot = mpf.make_addplot(enter_long, type='scatter', marker='^', markersize=25, panel=0, color='g')

  plots = [up_plot, down_plot]
  mpf.plot(df, type='candle', style='yahoo', addplot=plots,  figsize=(20,10),)

In [ ]:
df = calculate_entries(symbol, lookhead=2500)
make_plot(df, margin=100)

TypeError: ignored